In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset='imdb'

from data.imdb_loader import DataLoader
# from data.youtube_loader import DataLoader
# from data.trec_loader import DataLoader
# from data.sms_loader import DataLoader
# from data.census_loader import DataLoader
dl = DataLoader()
_, _, _, train_ground, val_ground, test_ground,_,_,_, train_text, val_text, test_text = dl.load_data(dataset=dataset)
# _, _, _, train_ground, val_ground, test_ground, _,_,_,train_text, val_text, test_text= dl.load_data(dataset=dataset)

len(labels 1920


In [3]:
fname = './LFs/'+dataset+'/val_5_dict_dt1/normal_reef.npy'
train_reef = np.load(fname)

In [4]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [5]:

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [6]:
df_train = pd.DataFrame(train_text)[0:83]

In [7]:
tokenized = df_train[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

train_features = last_hidden_states[0][:,0,:].numpy()

In [8]:
# test_text= np.delete(test_text , (138), axis=0) # For Youtube dataset

In [9]:
# test_ground = np.delete(test_ground , (138), axis=0) # For Youtube dataset

In [10]:
train_features.shape

(83, 768)

In [11]:
df_test = pd.DataFrame(test_text)
tokenized = df_test[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
k = 0
for i in tokenized.values:
    if len(i) > 500:
        print(len(i),k)
    k+=1
    if len(i) > max_len:
        max_len = len(i)
print('max len ', max_len)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

test_features = last_hidden_states[0][:,0,:].numpy()

max len  95


In [12]:
train_reef = train_reef[:83]

In [13]:
from lstm.lstm import *
f1_all = []
pr_all = []
re_all = []
val_acc_all = []


bs_arr = [64]#,128,256]
n_epochs_arr = [5]#,10,25]

for bs in bs_arr:
    for n in n_epochs_arr:
        y_pred = lstm_simple(train_features, train_reef, test_features, test_ground, bs=bs, n=n)
        predictions = np.round(y_pred)
        
        val_acc_all.append(np.sum(predictions == test_ground)/float(np.shape(test_ground)[0]))
        f1_all.append(metrics.f1_score(test_ground, predictions))
        pr_all.append(metrics.precision_score(test_ground, predictions))
        re_all.append(metrics.recall_score(test_ground, predictions))

Using TensorFlow backend.
W0512 15:27:04.726302 140684418361152 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0512 15:27:04.739998 140684418361152 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0512 15:27:04.753310 140684418361152 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0512 15:27:04.755266 140684418361152 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform 

before  (83, 768)
after (83, 768, 1)


W0512 15:27:05.931015 140684418361152 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0512 15:27:05.936126 140684418361152 deprecation.py:323] From /home/ayusham/.local/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0512 15:27:06.649588 140684418361152 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 83 samples, validate on 500 samples
Epoch 1/5
83/83 [==============================] - 16s 193ms/step - loss: 0.6932 - acc: 0.0000e+00 - val_loss: 0.6861 - val_acc: 0.5260
Epoch 2/5
83/83 [==============================] - 16s 198ms/step - loss: 0.6873 - acc: 0.0000e+00 - val_loss: 0.6851 - val_acc: 0.5260
Epoch 3/5
83/83 [==============================] - 15s 185ms/step - loss: 0.6816 - acc: 0.0000e+00 - val_loss: 0.6859 - val_acc: 0.5260
Epoch 4/5
83/83 [==============================] - 14s 174ms/step - loss: 0.6850 - acc: 0.0000e+00 - val_loss: 0.6869 - val_acc: 0.5260
Epoch 5/5
83/83 [==============================] - 15s 176ms/step - loss: 0.6836 - acc: 0.0000e+00 - val_loss: 0.6888 - val_acc: 0.5260
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 768)               2365440   
_________________________________________________________________
dense_1 (De

NameError: name 'metrics' is not defined

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_ground[0:83])

In [ ]:
lr_clf.score(test_features, test_ground)

In [ ]:
test_ground.shape

In [ ]:
pred = lr_clf.predict(test_features)

In [ ]:
from sklearn import metrics
print('Accuracy : ', np.sum(pred==test_ground)/len(test_ground))
print ('Test F1 Score: ', metrics.f1_score(test_ground, pred, average='binary'))
print ('Test Precision: ', metrics.precision_score(test_ground, pred, average='binary'))
print ('Test Recall: ', metrics.recall_score(test_ground, pred, average='macro'))